# Avito Demand Prediction Challenge - LightGBM Model

## Introduction
### The challenge
When selling used goods online, a combination of tiny, nuanced details in a product description can make a big difference in drumming up interest.

Avito, Russia’s largest classified advertisements website, is deeply familiar with this problem. Sellers on their platform sometimes feel frustrated with both too little demand (indicating something is wrong with the product or the product listing) or too much demand (indicating a hot item with a good description was underpriced).

In [their fourth Kaggle competition](https://kaggle.com/c/avito-demand-prediction), Avito is challenging participants to predict demand for an online advertisement based on its full description (title, description, images, etc.), its context (geographically where it was posted, similar ads already posted) and historical demand for similar ads in similar contexts. With this information, Avito can inform sellers on how to best optimize their listing and provide some indication of how much interest they should realistically expect to receive.

The description of the data files from the [data page](https://www.kaggle.com/c/avito-demand-prediction/data):

* train.csv - Train data.
* test.csv - Test data. Same schema as the train data, minus deal_probability.
* train_active.csv - Supplemental data from ads that were displayed during the same period as train.csv. Same schema as the train data, minus deal_probability.
* test_active.csv - Supplemental data from ads that were displayed during the same period as test.csv. Same schema as the train data, minus deal_probability.
* periods_train.csv - Supplemental data showing the dates when the ads from train_active.csv were activated and when they where displayed.
* periods_test.csv - Supplemental data showing the dates when the ads from test_active.csv were activated and when they where displayed. Same schema as periods_train.csv, except that the item ids map to an ad in test_active.csv.
* train_jpg.zip - Images from the ads in train.csv.
* test_jpg.zip - Images from the ads in test.csv.
* sample_submission.csv - A sample submission in the correct format.

### LightGBM model

In this notebook, we will train a [LightGBM](https://github.com/Microsoft/LightGBM) model to predict deal probability. LightGBM is a fast, distributed, high performance gradient boosting framework based on decision tree algorithms. It is under the umbrella of the [DMTK](http://github.com/microsoft/dmtk) project of Microsoft.

We break this notebook down into 5 steps.

- [Step 1](#step1): Read in csv files
- [Step 2](#step2): Engineer features: 
    - [categorical features](#step2a)
    - [numerical features](#step2b)
    - [text](#step2b)
- [Step 3](#step3): Perform Ridge Regression and k-fold cross validation
- [Step 4](#step4): Train and validate the model
- [Step 5](#step5): Predict deal probabilities

First, let's import some libraries.

In [1]:
import numpy as np
import pandas as pd

import gc
import random
random.seed(2018)

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix

from nltk.corpus import stopwords 

import lightgbm as lgb

import matplotlib.pyplot as plt

import string

from utils import *

%matplotlib inline

/home/trang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/trang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<a id='step1'></a>
## Load data

We will load the training, testing and aggregated data.

In [2]:
train = pd.read_csv("./csv/train.csv", index_col = "item_id", parse_dates = ["activation_date"])
test = pd.read_csv("./csv/test.csv", index_col = "item_id", parse_dates = ["activation_date"])
train.head()

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
item_id,,,,,,,,,,,,,,,,,
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [3]:
# Load and merge aggregated features into training and testing dataframes
af = pd.read_csv("csv/aggregated_features.csv", index_col=False)
train = train.merge(af, on="user_id", how="left")
test = test.merge(af, on="user_id", how="left")

agg_cols = list(af.columns)[1:]

del af
gc.collect()

54

We will combine training and testing data so that it's convenient when engineering features, etc.

In [4]:
# Make a copy of the target column before merging
y_train = train.deal_probability.copy()
train.drop("deal_probability", axis=1, inplace=True)

print("Train shape:", train.shape)
print("Test shape:", test.shape)

# Store the number of rows for each dataframe
n_train_rows = train.shape[0]
n_test_rows = test.shape[0]

# Combine traing and testing data
df = pd.concat([train, test], axis=0)

# Remove unused features
df.drop(["activation_date", "image"],axis=1,inplace=True)
print("All data shape:", df.shape)

del train, test
gc.collect()

Train shape: (1503424, 19)
Test shape: (508438, 19)
All data shape: (2011862, 19)


<a id='step2'></a>
## Engineer features
<a id='step2a'></a>
### Categorical features

We will first engineer categorical features: filling NA values and encoding the features.

In [ ]:
categorical = ["user_id", "region","city", "parent_category_name", "category_name", "user_type", "image_top_1", "param_1", "param_2", "param_3"]
# Fill NA values for image_top_1 with -1
df["image_top_1"].fillna(-1, inplace=True)

# Encode the features using LabelEncoder since we are using a decision tree model.
# If we use other models, we should consider One-Hot-Encoding + PCA
# https://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor
label_encoder = preprocessing.LabelEncoder()
for col in categorical:
    df[col].fillna("unknown")
    df[col] = label_encoder.fit_transform(df[col].astype(str))

<a id='step2b'></a>
### Numerical features

We will move on to numerical features. 
- For prices, we will take the natural logarithm of price for reasons describe [here](https://stats.stackexchange.com/questions/298/in-linear-regression-when-is-it-appropriate-to-use-the-log-of-an-independent-va). Then we will fill NA prices with the median value. 
- For aggregated features, we will simply fill NA values with -1

In [ ]:
df["price"] = np.log1p(df["price"])
df["price"].fillna(df.price.median(), inplace=True)
for col in agg_cols:
    df[col].fillna(-1, inplace=True)

<a id='step2c'></a>
### Text features

Now we will engineer text features. 
- For prices, we will take the natural logarithm of price for reasons describe [here](https://stats.stackexchange.com/questions/298/in-linear-regression-when-is-it-appropriate-to-use-the-log-of-an-independent-va). Then we will fill NA prices with the median value. 
- For aggregated features, we will simply fill NA values with -1